In [ ]:
import os
import re
import pandas as pd

# paths
BASE_DIR = os.getcwd()
FILES_DIR = os.path.join(BASE_DIR, "Files")

IDENTIFIER_PATH = os.path.join(BASE_DIR, "identifier_match.xlsx")
TRANSACTIONS_PATH = os.path.join(BASE_DIR, "overview_transactions.xlsx")

OUTPUT_PATH = os.path.join(BASE_DIR, "matched_rics.csv")

# load excel flies
identifier_df = pd.read_excel(IDENTIFIER_PATH)
transactions_df = pd.read_excel(TRANSACTIONS_PATH)

# normalize column names
identifier_df.columns = identifier_df.columns.str.strip()
transactions_df.columns = transactions_df.columns.str.strip()

# extract unique RICs from filenames
ric_pattern = re.compile(r"-([A-Z0-9\.^]+)-\d+-Brief\.txt$")

rics = set()

for fname in os.listdir(FILES_DIR):
    match = ric_pattern.search(fname)
    if match:
        rics.add(match.group(1))

print(f"Extracted {len(rics)} unique RICs from filenames")

# prepare result rows
rows = []

for ric in sorted(rics):

    # match in identifier_match.xlsx
    id_match = identifier_df[identifier_df["RIC"] == ric]
    if id_match.empty:
        continue

    # match in overview_transactions.xlsx (RIC1 only)
    tx_match = transactions_df[transactions_df["RIC1"] == ric]
    if tx_match.empty:
        continue

    # take first match
    id_row = id_match.iloc[0]
    tx_row = tx_match.iloc[0]

    rows.append({
        "RIC": ric,
        "ISIN": id_row.get("ISIN"),
        "CUSIP": id_row.get("CUSIP"),
        "AcquirorName": tx_row.get("Acquiror Name"),
        "TargetName": tx_row.get("Target Name"),
        "AnnounceDate": tx_row.get("Announce Date")
    })

# create output CSV
result_df = pd.DataFrame(rows)

result_df.to_csv(OUTPUT_PATH, index=False)

print(f"Saved {len(result_df)} matched RICs to {OUTPUT_PATH}")

Extracted 1239 unique RICs from filenames
Saved 1177 matched RICs to c:\Users\bulju\OneDrive\Desktop\Master\3 Semester\Data Science in Business 2\Assignment 1\dsb2-assignment1\matched_rics.csv


Getting stock prices for all companies from Compustat

In [ ]:
import pandas as pd
import numpy as np
import wrds
import time

# load matched_rics.csv
events = pd.read_csv("matched_rics.csv", parse_dates=["AnnounceDate"])
events["CUSIP"] = events["CUSIP"].astype(str).replace({"nan": np.nan, "None": np.nan})
events["ISIN"] = events["ISIN"].astype(str).replace({"nan": np.nan, "None": np.nan})

# connect to wrds
db = wrds.Connection()

# 1 build identifier -> gvkey/iid map CUSIP preferred else ISIN
sec = db.raw_sql("""
    SELECT gvkey, iid, cusip, isin
    FROM comp.security
""")

# normalize
sec["cusip"] = sec["cusip"].astype(str).str.strip().replace({"nan": np.nan})
sec["isin"]  = sec["isin"].astype(str).str.strip().replace({"nan": np.nan})

events_map = events.copy()
events_map["CUSIP"] = events_map["CUSIP"].astype(str).str.strip().replace({"nan": np.nan, "None": np.nan})
events_map["ISIN"]  = events_map["ISIN"].astype(str).str.strip().replace({"nan": np.nan, "None": np.nan})

# two mapping tables, then stack with priority
cusip_map = (
    sec.dropna(subset=["cusip"])
       .rename(columns={"cusip": "id"})
       .assign(id_type="CUSIP")[["id_type", "id", "gvkey", "iid"]]
)

isin_map = (
    sec.dropna(subset=["isin"])
       .rename(columns={"isin": "id"})
       .assign(id_type="ISIN")[["id_type", "id", "gvkey", "iid"]]
)

id_map = pd.concat([cusip_map, isin_map], ignore_index=True)

# create one "preferred id" per event: CUSIP if present else ISIN
events_map["id_type"] = np.where(events_map["CUSIP"].notna() & (events_map["CUSIP"] != ""), "CUSIP", "ISIN")
events_map["id"] = np.where(events_map["id_type"] == "CUSIP", events_map["CUSIP"], events_map["ISIN"])

mapped = events_map.merge(
    id_map,
    on=["id_type", "id"],
    how="left"
)

print("Mapped rows:", mapped["gvkey"].notna().sum(), "/", len(mapped))
print("Unmapped examples:")
print(mapped.loc[mapped["gvkey"].isna(), ["RIC", "CUSIP", "ISIN"]].head(10))

# 2 oull daily prices : efficient + progress
mapped_ok = mapped.dropna(subset=["gvkey", "iid"]).copy()

# build (gvkey,iid) date ranges to query
mapped_ok["start"] = mapped_ok["AnnounceDate"] - pd.Timedelta(days=300)  # estimation buffer
mapped_ok["end"]   = mapped_ok["AnnounceDate"] + pd.Timedelta(days=30)   # event + a bit after

pairs = mapped_ok[["gvkey", "iid", "start", "end", "RIC"]].copy()

print(f"Unique events with gvkey/iid: {len(pairs)} / {len(mapped)}")

# wrds postgres has a limit on query length
#query in batches and print progress
CHUNK_SIZE = 50

all_prices = []
t0 = time.time()

# helper to format values safely for SQL IN
def sql_list(vals):
    return ",".join([f"'{v}'" for v in vals])

for i in range(0, len(pairs), CHUNK_SIZE):
    chunk = pairs.iloc[i:i+CHUNK_SIZE].copy()

    # use the min/max dates for the chunk then filter in pandas by event window
    min_date = chunk["start"].min().strftime("%Y-%m-%d")
    max_date = chunk["end"].max().strftime("%Y-%m-%d")

    # build tuple list for (gvkey,iid)
    tuple_list = ",".join([f"('{g}','{iid}')" for g, iid in zip(chunk["gvkey"], chunk["iid"])])

    query = f"""
        SELECT gvkey, iid, datadate, prccd
        FROM comp.secd
        WHERE (gvkey, iid) IN ({tuple_list})
          AND datadate BETWEEN '{min_date}' AND '{max_date}'
          AND prccd IS NOT NULL
    """

    q_start = time.time()
    prices_chunk = db.raw_sql(query)
    q_end = time.time()

    prices_chunk["datadate"] = pd.to_datetime(prices_chunk["datadate"])
    all_prices.append(prices_chunk)

    done = min(i + CHUNK_SIZE, len(pairs))
    elapsed = time.time() - t0
    avg_per_event = elapsed / done
    eta = avg_per_event * (len(pairs) - done)

    print(
        f"[{done}/{len(pairs)}] chunk_rows={len(prices_chunk):,} "
        f"chunk_time={q_end-q_start:.2f}s | elapsed={elapsed:.1f}s | ETA~{eta:.1f}s"
    )

prices = pd.concat(all_prices, ignore_index=True) if all_prices else pd.DataFrame()
print("Total price rows pulled:", len(prices))

# merge and filter per event calendar window precisely 30 day buffer
merged = mapped_ok.merge(prices, on=["gvkey", "iid"], how="left")

wide = merged[
    (merged["datadate"] >= merged["start"]) &
    (merged["datadate"] <= merged["end"])
].copy()

import statsmodels.api as sm

# 1 compute returns
wide = wide.sort_values(["RIC", "datadate"])
wide["ret"] = wide.groupby("RIC")["prccd"].pct_change()

# 2 pull ff3 daily factors from wrds
try:
    ff3 = db.raw_sql("""
        SELECT date, mktrf, smb, hml, rf
        FROM ff.factors_daily
    """)
except Exception:
    ff3 = db.raw_sql("""
        SELECT date, mktrf, smb, hml, rf
        FROM ff.factors_daily_3
    """)

ff3["date"] = pd.to_datetime(ff3["date"])

# convert % -> decimals if needed
if ff3["rf"].abs().median() > 0.001:
    for c in ["mktrf", "smb", "hml", "rf"]:
        ff3[c] = ff3[c] / 100.0

# 3 merge factors into your panel
panel = wide.merge(ff3, left_on="datadate", right_on="date", how="inner").drop(columns=["date"])

# 4 convenience columns
panel["rm"] = panel["mktrf"] + panel["rf"]          # market return
panel["ri_minus_rf"] = panel["ret"] - panel["rf"]   # excess stock return
panel["rm_minus_rf"] = panel["rm"] - panel["rf"]    # equals mktrf if constructed this way

# 5save one master dataset
panel.to_csv("panel_prices_factors.csv", index=False)
print("Saved: panel_prices_factors.csv | rows:", len(panel), "| events:", panel["RIC"].nunique())

# for each event RIC, take nearest 5 trading days before and 5 after (incl event day when possible)
def slice_trading_window(df):
    ad = df["AnnounceDate"].iloc[0]
    df = df.dropna(subset=["datadate"]).sort_values("datadate")
    before = df[df["datadate"] < ad].tail(5)
    on_after = df[df["datadate"] >= ad].head(6)
    return pd.concat([before, on_after], axis=0)

event_window = (
    panel.groupby("RIC", group_keys=False)  # use panel here
         .apply(slice_trading_window)
         .reset_index(drop=True)
)

event_window.to_csv("panel_event_window_prices_factors.csv", index=False)
print("Saved: panel_event_window_prices_factors.csv")
print("Rows:", len(event_window), "| Events:", event_window["RIC"].nunique())

# output
out_cols = ["RIC", "ISIN", "CUSIP", "AcquirorName", "TargetName", "AnnounceDate",
            "gvkey", "iid", "datadate", "prccd"]

event_window[out_cols].to_csv("compustat_event_window_prices.csv", index=False)

print("Saved:", "compustat_event_window_prices.csv")
print("Rows:", len(event_window))


WRDS recommends setting up a .pgpass file.
pgpass file created at C:\Users\bulju\AppData\Roaming\postgresql\pgpass.conf
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Mapped rows: 1153 / 1228
Unmapped examples:
             RIC      CUSIP          ISIN
2        2353.TW        NaN  TW0002353000
3         4578.T        NaN  JP3188220002
4      600938.SS        NaN  CNE100005980
5         6753.T        NaN  JP3359600008
6         6857.T        NaN  JP3122400009
7        9961.HK        NaN  KYG9066F1019
14        ABDN.L        NaN  GB00BF8Q6K64
24     ACV.N^K06  013068101  US0130681010
40    AFRE.L^H15        NaN  GB00B0672758
64  ALTEO.PK^D08  020039707  US0200397075
Unique events with gvkey/iid: 1153 / 1228
[50/1153] chunk_rows=133,810 chunk_time=11.38s | elapsed=11.4s | ETA~251.6s
[100/1153] chunk_rows=140,628 chunk_time=13.20s | elapsed=24.6s | ETA~259.3s
[150/1153] chunk_rows=189,216

1b and 1c

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

panel = pd.read_csv("panel_prices_factors.csv", parse_dates=["AnnounceDate", "datadate"])

# ensure sorted
panel = panel.sort_values(["RIC", "datadate"]).copy()

# compute tau (trading-day index relative to event day) using the event-window file later,
# but for estimation we’ll filter by dates around AnnounceDate.

def fit_models_and_ar(one_event):
    """
    one_event = all rows for one (RIC, AnnounceDate) from panel_prices_factors
    returns a dataframe of event-window rows with AR/CAR for Market Model and FF3
    """
    ric = one_event["RIC"].iloc[0]
    ad = one_event["AnnounceDate"].iloc[0]

    # define estimation window and event window by TRADING DAYS via sorting
    one_event = one_event.dropna(subset=["ret", "mktrf", "smb", "hml", "rf"]).sort_values("datadate")

    # create an integer index relative to announcement date based on trading days present in data
    # tau=0 is the first trading day with datadate >= AnnounceDate
    idx0 = one_event.index[one_event["datadate"] >= ad]
    if len(idx0) == 0:
        return None  # no event day trading data
    idx0 = idx0[0]

    # map each row to tau using position, not calendar days
    pos = np.arange(len(one_event))
    pos0 = one_event.index.get_loc(idx0)
    one_event = one_event.reset_index(drop=True)
    one_event["tau"] = np.arange(len(one_event)) - pos0

    # estimation window: [-250, -30]
    est = one_event[(one_event["tau"] >= -250) & (one_event["tau"] <= -30)].copy()
    evw = one_event[(one_event["tau"] >= -5) & (one_event["tau"] <= 5)].copy()

    # need enough observations to fit
    if len(est) < 60:   # rule of thumb
        return None

    # Market model: ret ~ const + rm
    # rm = mktrf + rf (market return)
    est["rm"] = est["mktrf"] + est["rf"]
    evw["rm"] = evw["mktrf"] + evw["rf"]

    X_mm = sm.add_constant(est["rm"])
    y_mm = est["ret"]
    mm = sm.OLS(y_mm, X_mm).fit()

    evw["exp_mm"] = mm.predict(sm.add_constant(evw["rm"]))
    evw["ar_mm"] = evw["ret"] - evw["exp_mm"]
    evw["car_mm"] = evw["ar_mm"].cumsum()

    # FF3: (ret - rf) ~ const + mktrf + smb + hml
    est["ri_minus_rf"] = est["ret"] - est["rf"]
    evw["ri_minus_rf"] = evw["ret"] - evw["rf"]

    X_ff = sm.add_constant(est[["mktrf", "smb", "hml"]])
    y_ff = est["ri_minus_rf"]
    ff = sm.OLS(y_ff, X_ff).fit()

    evw["exp_ff3_excess"] = ff.predict(sm.add_constant(evw[["mktrf", "smb", "hml"]]))
    evw["ar_ff3"] = evw["ri_minus_rf"] - evw["exp_ff3_excess"]
    evw["car_ff3"] = evw["ar_ff3"].cumsum()

    # keep identifiers + outputs
    keep_cols = [
        "RIC","AcquirorName","TargetName","AnnounceDate","datadate","tau",
        "prccd","ret",
        "ar_mm","car_mm",
        "ar_ff3","car_ff3"
    ]
    return evw[keep_cols]

# apply per event (RIC+AnnounceDate)
out = []
for (ric, ad), g in panel.groupby(["RIC", "AnnounceDate"]):
    res = fit_models_and_ar(g)
    if res is not None:
        out.append(res)

event_ar = pd.concat(out, ignore_index=True) if out else pd.DataFrame()
event_ar.to_csv("event_AR_CAR_mm_ff3.csv", index=False)

print("Saved: event_AR_CAR_mm_ff3.csv")
print("Events with results:", event_ar[["RIC","AnnounceDate"]].drop_duplicates().shape[0])
print("Rows:", len(event_ar))

Saved: event_AR_CAR_mm_ff3.csv
Events with results: 906
Rows: 9966


## Event Date, Event Window, and Estimation Window

To analyze stock price reactions to merger and acquisition (M&A) announcements, an event-study framework is applied. This requires specifying an event date, an event window, and an estimation window in a transparent and theoretically motivated way.

### Event Date
The event date is defined as the **public announcement date of the M&A transaction**, as reported in the Reuters transaction database. This date represents the moment when new, value-relevant information becomes available to the market. Under the semi-strong form of the Efficient Market Hypothesis, stock prices are expected to adjust rapidly once such information is publicly disclosed, making the announcement date the natural focal point for measuring abnormal returns.

### Event Window
The event window is set to **[-5, +5] trading days** around the announcement date.

This window is chosen to capture not only the immediate market reaction on the announcement day, but also potential dynamics surrounding the event:
- The pre-announcement period \([-5, -1]\) allows for the possibility of **information leakage**, rumors, or anticipatory trading prior to the official announcement.
- The post-announcement period \([+1, +5]\) accounts for **delayed market reactions**, gradual information processing, and potential effects of thin trading or liquidity constraints.

Using a symmetric window around the event date is standard practice in the event-study literature and provides a balanced view of short-term market responses.

### Estimation Window
The estimation window used to estimate normal (expected) returns is defined as **[-250, -30] trading days** relative to the announcement date.

This window is sufficiently long to obtain reliable estimates of risk parameters (e.g., betas in the market model and the Fama–French three-factor model), while being far enough from the event itself to avoid contamination from event-related information. The exclusion of the period immediately preceding the event ensures that parameter estimates are not influenced by anticipation effects or abnormal trading behavior related to the upcoming announcement.

Overall, the chosen event date and windows follow established conventions in empirical finance and provide a robust basis for measuring abnormal and cumulative abnormal returns associated with M&A announcements.